# RSSI region around Base Station

In [23]:
import pandas as pd
import numpy as np
import os
gcpath = "../../data/2g_gongcan_new.csv"
gongcan = pd.read_csv(gcpath)
trainPath = "../../data/train/forward.csv"
trainDF = pd.read_csv(trainPath)

In [3]:
gongcan = gongcan[['LAC','CI','longitude','latitude']]

In [5]:
trainDF = trainDF.merge(gongcan, left_on=['All-LAC', 'All-Cell Id'], right_on=['LAC', 'CI'], how='left')

In [15]:
trainDF = trainDF[['All-Longitude','All-Latitude','longitude','latitude','All-RxLev Sub (dBm)']]


array([[ 121.4428333 ,   31.193475  ,           nan,           nan,  -64.        ],
       [ 121.4428333 ,   31.193475  ,           nan,           nan,  -62.        ],
       [ 121.4428333 ,   31.193475  ,  121.445424  ,   31.194791  ,  -64.        ],
       ..., 
       [ 121.448195  ,   31.19626333,  121.446515  ,   31.199551  ,  -68.        ],
       [ 121.448195  ,   31.19626167,  121.446515  ,   31.199551  ,  -74.        ],
       [ 121.448195  ,   31.19626167,  121.446515  ,   31.199551  ,  -71.        ]])

In [19]:
trainDF.to_csv("out.csv",header=False,index=False)

In [94]:
import re
import urllib2
import json
def write_batch(buffer_list, buffer_lon, buffer_lat, f_correct_route):
	base_url = 'http://api.zdoz.net/transmore.ashx?'
	url = base_url \
	   + 'lats=' + ';'.join(buffer_lat) + '&' \
	   + 'lngs=' + ';'.join(buffer_lon) + '&' \
	   + 'type=1'
	req = urllib2.Request(url)
	response = urllib2.urlopen(req)
	res = response.read()
	print url
	# print res
	res = res[res.find('[')+1:res.find(']')]
	res = res.split('},')
	lonlat = []
	for i in xrange(len(buffer_list)):
		lonlat = []
		for j in xrange(2):
			k = i*2+j;
			try:
				if k == len(res)-1:
					lonlat.append(json.loads(res[k]))
				else:
					lonlat.append(json.loads(res[k]+'}'))
			except:
				print k,len(res)
				print i,j		

		buffered_line = buffer_list[i]	
		f_correct_route.write(str(lonlat[0]['Lng'])+','+str(lonlat[0]['Lat'])+','+\
			str(lonlat[1]['Lng'])+','+str(lonlat[1]['Lat'])+','+buffered_line[4]+'\n')

		# f_correct_route.write(buffered_line[0]+','+buffered_line[1]+','+buffered_line[2]+','+buffered_line[3]+','+\
		#    str(lonlat['Lat'])+'N,'+str(lonlat['Lng'])+'E,'+buffered_line[6]+','+buffered_line[7]+','+buffered_line[8]+','+buffered_line[9]+'\n')

def correct_route(filename, f, files):
	add_prefix = re.compile('\.csv')
	if f.find('.csv') == 0:
		return
	if add_prefix.sub('_corrected.csv', f) in files or f.find('corrected') > 0:
	    return
	print f
	f_raw_route = open(filename)
	f_correct_route = open(add_prefix.sub('_corrected.csv', filename), 'w')

	base_url = 'http://api.zdoz.net/transmore.ashx?'
	data = {}

# 	head_string = f_raw_route.readline()
# 	f_correct_route.write(head_string)

	buffer_list = []
	buffer_lat = []
	buffer_lon = []
	for line in f_raw_route:
		line = line.strip().split(',') 
		line = ['0' if s=='' else s  for s in line]	

		if len(buffer_list) == 20:
			write_batch(buffer_list, buffer_lon, buffer_lat, f_correct_route)
			buffer_list = []
			buffer_lon = []
			buffer_lat = []
# 			break

		buffer_list.append(line)	
		buffer_lon.append(line[0].strip())
		buffer_lat.append(line[1].strip())
		buffer_lon.append(line[2].strip())
		buffer_lat.append(line[3].strip())

    

	write_batch(buffer_list, buffer_lon, buffer_lat, f_correct_route)


	f_raw_route.close()
	f_correct_route.close()
	print filename
	# os._exit(0)
files = os.listdir('.')


In [ ]:
correct_route('out.csv','out.csv',files)

out.csv
http://api.zdoz.net/transmore.ashx?lats=31.193475;0;31.193475;0;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;0;31.193475;0;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.193475;31.194791;31.19347833;31.194791;31.19347833;31.194791;31.19347833;31.194791&lngs=121.4428333;0;121.4428333;0;121.4428333;121.445424;121.4428333;121.445424;121.4428333;121.445424;121.4428333;121.445424;121.4428333;121.445424;121.4428333;121.445424;121.442835;121.445424;121.442835;121.445424;121.442835;121.445424;121.442835;0;121.442835;0;121.442835;121.445424;121.442835;121.445424;121.442835;121.445424;121.442835;121.445424;121.4428433;121.445424;121.4428433;121.445424;121.4428433;121.445424&type=1
http://api.zdoz.net/transmore.ashx?lats=31.19347833;31.194791;31.19347833;31.194791;31.19347833;31.194791;31.19347833;31.194791;31.19347833;31.194791;3